## Optimization Program for Venue Series
#### Copyright © 2024 Hoshino Math Services

In [1]:
# Import the Python Modules.

import time
import math
import numpy as np
import pandas as pd
import csv
from ortools.linear_solver import pywraplp

AttendeeFile = pd.read_excel('Attendee List (Vendor Rated).xlsx')
VendorRatingInfo = AttendeeFile.values.tolist()

AttendeeList = []
FullAttendeeList = []

for i in range(len(AttendeeFile)):
    AttendeeName = AttendeeFile["FIRST"][i] + " " + AttendeeFile["LAST"][i]
    if AttendeeName in FullAttendeeList:
            print("ERROR", AttendeeName, "appears twice")
    else:
        FullAttendeeList.append(AttendeeName)
    if AttendeeFile["Checked-in"][i] == 'x':
        AttendeeList.append(AttendeeName)
            
VendorFile = pd.read_excel('Vendor List.xlsx')
VendorList = []
for i in range(len(VendorFile)):
    if VendorFile["ARRIVED"][i] == 'x':
        VendorName = VendorFile["COMPANY"][i]
        if VendorName in VendorList:
            print("ERROR", VendorName, "appears twice")
        else:
            VendorList.append(VendorName)

AttendeeRatingFile = pd.read_excel('Attendee Form.xlsx')


V = len(VendorList)
A = len(AttendeeList)

print("There are", V, "vendors and", A, "attendees that are present")

There are 21 vendors and 48 attendees that are present


In [2]:
for i in range(len(AttendeeRatingFile)):
    AttendeeName = AttendeeRatingFile["FIRST"][i] + " " + AttendeeRatingFile["LAST"][i] 
    if not AttendeeName in FullAttendeeList:
        print(AttendeeName, "does not appear in the list of Attendees")

In [3]:
# Determine the score of Vendor V being matched with Attendee A

VAScore = [[0 for a in range(A)] for v in range(V)]
VAScore1 = [[1 for a in range(A)] for v in range(V)]
VAScore2 = [[1 for a in range(A)] for v in range(V)]


# Assign 3 points for each occurrence of an Attendee picking a Vendor
# Assign 2 points for each occurrence of a Vendor picking an Attendee

for i in range(len(AttendeeRatingFile)):
    AttendeeName = AttendeeRatingFile["FIRST"][i] + " " + AttendeeRatingFile["LAST"][i]
    if AttendeeName in AttendeeList:
        AttendeeIndex = AttendeeList.index(AttendeeName)
        for VendorName in VendorList:
            VendorIndex = VendorList.index(VendorName)
            Response = AttendeeRatingFile[VendorName][i]
            if Response == 'Very Interested.': Score = 2
            elif Response == 'Not Interested.': Score = 0
            elif Response == 'Interested.': Score = 1
            else: print("ERROR", Response)
            VAScore1[VendorIndex][AttendeeIndex] = Score

for i in range(len(VendorRatingInfo)):
    AttendeeName = VendorRatingInfo[i][1] + " " + VendorRatingInfo[i][2]
    if AttendeeName in AttendeeList:
        AttendeeIndex = AttendeeList.index(AttendeeName)
        for v in range(len(VendorRatingInfo[0])-len(VendorList), len(VendorRatingInfo[0])):
            Score = VendorRatingInfo[i][v]
            VendorIndex = v-len(VendorRatingInfo[0])+len(VendorList)
            if Score in [0,1,2]: VAScore2[VendorIndex][AttendeeIndex] = Score
                
for v in range(V):
    for a in range(A):
        VAScore[v][a] = 3*VAScore1[v][a] + 2*VAScore2[v][a]

        
# Assign -100 points for each occurrence of (Vendor, Attendee) pair where neither side has picked the other

for v in range(V):
    for a in range(A):
        if VAScore[v][a] == 0:
            VAScore[v][a] = -100
            

In [4]:
# Assign -10,000 points for each occurrence of (Vendor, Attendee) pair where the Vendor is not Local 
# and the Attendee is not National, they cannot be paired   

NonLocalVendorList = []
for i in range(len(VendorFile)):
    VendorName = VendorFile["COMPANY"][i]
    if VendorName in VendorList:
        if VendorFile["STATUS"][i] == 'Not Local':
            NonLocalVendorList.append(VendorName)
                
NonNationalAttendeeList = []
for i in range(len(AttendeeFile)):
    AttendeeName = AttendeeFile["FIRST"][i] + " " + AttendeeFile["LAST"][i]
    if AttendeeName in AttendeeList:
        if not 'ational' in AttendeeFile["Where do your events take place?"][i]:
            NonNationalAttendeeList.append(AttendeeName)
            
for v in range(V):
    for a in range(A):
        if VendorList[v] in NonLocalVendorList and AttendeeList[a] in NonNationalAttendeeList:
            VAScore[v][a] =- 10000
        
        
# Determine the number of meeting rounds

R = 12

In [5]:
# Optimize the assignment of Attendees to Vendors

StartTime = time.time()

solver = pywraplp.Solver("Venue Series", pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Vendors = range(len(VendorList))
Attendees = range(len(AttendeeList))
Rounds = range(12)

x = {}
for v in Vendors:
    for a in Attendees:
        for r in Rounds:
            x[v,a,r] = solver.IntVar(0,1, "x[%d,%d,%d]" % (v,a,r))

        
# CONSTRAINT 1: Each Vendor has one Attendee Meeting per round
for v in Vendors:
    for r in Rounds:
        solver.Add(sum(x[v,a,r] for a in Attendees) == 1)
        
        
# CONSTRAINT 2: Each Attendee has at most one Vendor Meeting per round
for a in Attendees:
    for r in Rounds:
        solver.Add(sum(x[v,a,r] for v in Vendors) <= 1)
        
        
# CONSTRAINT 3: Each Attendee has approximately the same number of Vendor Meetings
for a in Attendees:
    solver.Add(sum(x[v,a,r] for v in Vendors for r in Rounds) >= math.floor(V*12/A))
    solver.Add(sum(x[v,a,r] for v in Vendors for r in Rounds) <= math.ceil(V*12/A))
    
    
# CONSTRAINT 4: Each (Vendor, Attendee) pair can only meet once
for v in Vendors:
    for a in Attendees:
        solver.Add(sum(x[v,a,r] for r in Rounds) <= 1)
    
    
# Run the optimization
solver.Maximize(solver.Sum(VAScore[v][a] * x[v,a,r] for v in Vendors for a in Attendees for r in Rounds))
sol = solver.Solve()

Solution = []
for v in Vendors:
    for a in Attendees:
        for r in Rounds:
            if x[v,a,r].solution_value()==1:
                Solution.append([v,a,r])

In [6]:
# Export to Excel

OurColumns = ["Attendee ID", "Attendee Name", "Attendee Email", "Attendee Company"]
for r in Rounds:
    OurColumns.append("Round " + str(r+1))

M = []
for i in range(len(AttendeeFile)):
    AllVendors = ["" for r in Rounds]
    AttendeeName = AttendeeFile["FIRST"][i] + " " + AttendeeFile["LAST"][i]
    AttendeeEmail = AttendeeFile["EMAIL"][i]
    AttendeeCompany = AttendeeFile["COMPANY"][i]
    if AttendeeName in AttendeeList:
        AttendeeID = AttendeeList.index(AttendeeName)
        for mypair in Solution:
            if mypair[1] == AttendeeID:
                AllVendors[mypair[2]] = "V"+str(mypair[0]+1)
        M += [["A"+str(AttendeeID+1), AttendeeName, AttendeeEmail, AttendeeCompany] + AllVendors[0:R]]

FinalMatrix = pd.DataFrame(M, columns=OurColumns)
FinalMatrix.to_csv("Output File - Attendee Assignments.csv", index=False, encoding="utf-8-sig")


OurColumns = ["Vendor ID", "Vendor Rep", "Vendor Email", "Vendor Name"]

for r in Rounds:
    OurColumns.append("Round " + str(r+1))

M = []
for i in range(len(VendorFile)):
    AllAttendees = ["" for r in Rounds]
    VendorName = VendorFile["COMPANY"][i]
    VendorRep = VendorFile["PRIMARY REP"][i]
    VendorEmail = VendorFile["EMAIL"][i]
    
    if VendorName in VendorList:
        VendorID = VendorList.index(VendorName)
        for mypair in Solution:
            if mypair[0] == VendorID:
                AllAttendees[mypair[2]] = "A"+str(mypair[1]+1)
                
    M += [["V"+str(VendorID+1), VendorRep, VendorEmail, VendorName] + AllAttendees[0:R]]

FinalMatrix = pd.DataFrame(M, columns=OurColumns)
FinalMatrix.to_csv("Output File - Vendor Assignments.csv", index=False, encoding="utf-8-sig")

In [7]:
# Output the final score with the total running time

TotalScore = 0
for temp in Solution:
    TotalScore += VAScore[temp[0]][temp[1]]
      
TotalTime = time.time() - StartTime
print("Optimization Complete in", round(TotalTime,4), "seconds")   
print("The total score is", TotalScore, "points")
print("")

# Output any matches where neither the vendor nor the attendee has selected the other

for temp in Solution:
    if VAScore[temp[0]][temp[1]] == -100:
        print("Vendor", VendorList[temp[0]], "and Attendee", AttendeeList[temp[1]], "did not select each other")
        
        
# Output any matches where neither the vendor is not Local and the attendee is not National

for temp in Solution:
    if VAScore[temp[0]][temp[1]] < -9000:
        print("Vendor", VendorList[temp[0]], "is not local and Attendee", AttendeeList[temp[1]],
              "is not national")

Optimization Complete in 4.7259 seconds
The total score is 1657 points



In [19]:
# For each vendor, print the number of matches with 6 or more points, followed
# by the number of matches with 1 to 5 points, followed by the number of matches
# with 0 points.

for v in Vendors:
    response = [0,0,0]
    for trip in Solution:
        if v==trip[0]:
            if VAScore[trip[0]][trip[1]]>5: response[0]+=1
            elif VAScore[trip[0]][trip[1]]<1: response[2]+=1
            else: response[1]+=1            
    print(VendorList[v], response)

Paradox Hotel Vancouver [12, 0, 0]
Tourism Nanaimo [10, 2, 0]
Delta Hotels by Marriott Calgary Downtown [4, 8, 0]
BC Hotel Association [8, 4, 0]
Corner Collection [4, 8, 0]
The Josie Hotel, Autograph Collection [4, 8, 0]
Meet in Penticton [3, 9, 0]
Live Nation [7, 5, 0]
Blue Horizon Hotel [8, 4, 0]
Event Regina (SK) [8, 4, 0]
World of Hyatt (Hyatt Regency Vancouver, Hyatt Place Kelowna) [7, 5, 0]
Aldesta Hotels & Resorts [8, 4, 0]
Discover Saskatoon [3, 9, 0]
Air Canada [5, 7, 0]
Tourism Whistler [8, 4, 0]
Grouse Mountain Resorts [11, 1, 0]
Tourism Kamloops [2, 10, 0]
Pacific Yacht Charters & PYC Catering [5, 7, 0]
Tourisme Montreal [4, 8, 0]
Four Points by Sheraton + Hampton Inn & Suites by Hilton Kelowna Airport [11, 1, 0]
Chair-man Mills Corp. [11, 1, 0]
